In [57]:
import pandas as pd
import csv

import os, os.path
import sys

# All chosen fields for survey 1
A grass hay field, "106433_WSDA_SF_2017", was missed from meeting PDF! how the hell did this happen?
- I had dropped grass hay from disagreement table! 

In [101]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

chosen_fields_set1 = pd.DataFrame()
for a_choice_sheet in choices_set_1_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)
    chosen_fields_set1 = pd.concat([chosen_fields_set1, a_choice_sheet])

In [102]:
choices_set_2_xl = pd.ExcelFile(perry_dir + "set_2_handPicked.xlsx")
choices_set_2_sheet_names = choices_set_2_xl.sheet_names  # see all sheet names

chosen_fields_set2 = pd.DataFrame()
for a_choice_sheet in choices_set_2_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_2_xl.parse(a_choice_sheet)
    chosen_fields_set2 = pd.concat([chosen_fields_set2, a_choice_sheet])

In [103]:
print (len(chosen_fields_set2.ID))
print (len(chosen_fields_set2.ID.unique()))

322
322


In [104]:
# # check if there are overlap between chosen fields in set 1 and set 2
# set1_uniqueIDs = list(chosen_fields_set1.ID.unique())
# set2_uniqueIDs = list(chosen_fields_set2.ID.unique())

# A = chosen_fields_set2[chosen_fields_set2.ID.isin(set1_uniqueIDs)]
# B = chosen_fields_set1[chosen_fields_set1.ID.isin(set2_uniqueIDs)]
# A.shape

# Set 1 - post meeting

In [105]:
import pandas as pd
expert_label_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
file_name =  "Expert_label_Consensus_Set1_Meeting.xlsx"
Labels_xls = pd.ExcelFile(expert_label_dir + file_name)
sheet_names = Labels_xls.sheet_names  # see all sheet names
meeting_Consensus = Labels_xls.parse()

needed_cols = ["ID", "Vote"]
meeting_Consensus = meeting_Consensus[needed_cols]
meeting_Consensus.head(2)

,ID,Vote
0,160546_WSDA_SF_2015,1
1,99837_WSDA_SF_2017,2


# There are some "unsures" even after meeting. Remove them

In [106]:
print (meeting_Consensus.Vote.unique())

meeting_Consensus = meeting_Consensus[meeting_Consensus.Vote.isin([1, 2])]
meeting_Consensus.Vote = meeting_Consensus.Vote.astype(int)
print (meeting_Consensus.Vote.unique())

[1 2 0 '0/2']
[1 2]


# Set 1, pre-meeting

In [107]:
ML_data_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
set1_premeeting_consensus = pd.read_csv(ML_data_dir + "set1_premeeting_consensus.csv")
set1_premeeting_consensus.head(2)

,ID,PerrysVote,AndrewsVote,TimsVote,PerryVCorrected,Vote
0,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1
1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,Single Crop,1


In [108]:
drop_cols = ['PerrysVote', 'AndrewsVote', 'TimsVote']
set1_premeeting_consensus.drop(labels=drop_cols, axis=1, inplace=True)

# Concatenate set 1 pre and post meeting labels

In [109]:
set_1_labels = pd.concat([meeting_Consensus, set1_premeeting_consensus[['ID', 'Vote']]])

# we need this shit, cuz some fields were requested to be discussed or had comment
set_1_labels.drop_duplicates(inplace=True) 
set_1_labels.shape

(285, 2)

# Read Set 2 pre-meeting consensus

In [110]:
set2_premeeting_consensus = pd.read_csv(ML_data_dir + "set2_premeeting_consensus.csv")
set2_premeeting_consensus.head(2)

,ID,PerryV,AndrewV,PerryVCorrected,Vote
0,148920_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,1
1,152722_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,1


In [111]:
set1_IDs = list(set_1_labels.ID.unique())

#
#    Get rid of repeated fields
#
set2_premeeting_consensus = set2_premeeting_consensus[~set2_premeeting_consensus.ID.isin(set1_IDs)]

In [119]:
train_labels = pd.concat([set_1_labels, set2_premeeting_consensus[['ID', 'Vote']]])
train_labels.reset_index(drop=True, inplace=True)
# train_labels.drop_duplicates(inplace=True)
train_labels.shape

(437, 2)

In [120]:
out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "train_labels.csv"
train_labels.to_csv(out_name, index = False)